In [54]:
import json
import websocket
import time
import pandas as pd
import numpy as np
import random
from IPython.display import clear_output
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth',300)

In [55]:

api_token = ""
app_id = 16303  # Replace with your app_id or leave as 1089 for testing.

ws_link = "wss://qa130.deriv.dev/websockets/v3?l=EN&app_id=16303&brand=deriv"

connection = websocket.create_connection(ws_link)

def get_symbol():
    request = {
  "active_symbols": "brief",
  "product_type": "basic"
}
    connection.send(json.dumps(request))

    res = connection.recv()
    if res is None:
        print("error")
    data = json.loads(res)
    if "error" in data:
        print("Error:", data["error"]["message"])
        connection.close()
        
    elif data["msg_type"] == "active_symbols":
        print(data["active_symbols"])
        return data["active_symbols"]
            

def proposal_id(amount, symbol):
    request_proposal= {
        "amount": amount,
        "basis": "stake",
        "contract_type": "MULTUP",
        "multiplier" : 50,
        "currency": "USD",
        "proposal": 1,
        "req_id": 16,
        "subscribe": 1,
        "symbol": symbol
        #,"limit_order":{"take_profit": amount*2}
    }
    connection.send(json.dumps(request_proposal))
    while True:
        res = connection.recv()
        if res is None:
            break
        data = json.loads(res)
        if "error" in data:
            print("Error:", data["error"]["message"])
            connection.close()
            break
        elif data["msg_type"] == "proposal":
            return data['proposal']
            #return(data['proposal']["id"])

def auth_token(api_token):
    request_authorize = {
    "authorize": f"{api_token}"
    }
    connection.send(json.dumps(request_authorize))

    res = connection.recv()
    if res is None:
        print("error")
    data = json.loads(res)
    if "error" in data:
        print("Error:", data["error"]["message"])
        connection.close()
        
    elif data["msg_type"] == "authorize":
        print(data["authorize"])
        
            

def proposal_id(amount, symbol):
    request_proposal= {
        "amount": amount,
        "basis": "stake",
        "contract_type": "MULTUP",
        "multiplier" : 50,
        "currency": "USD",
        "proposal": 1,
        "req_id": 16,
        "subscribe": 1,
        "symbol": symbol
        #,"limit_order":{"take_profit": amount*2}
    }
    connection.send(json.dumps(request_proposal))
    while True:
        res = connection.recv()
        if res is None:
            break
        data = json.loads(res)
        if "error" in data:
            print("Error:", data["error"]["message"])
            connection.close()
            break
        elif data["msg_type"] == "proposal":
            return data['proposal']
            #return(data['proposal']["id"])

def buy_contract(amount, symbol, contract_type):
    request = {
        "buy": "1",
        "parameters": {
            "amount": amount,
            "app_markup_percentage": "0",
            "basis": "stake",
            "contract_type": contract_type,
            "currency": "USD",
            "multiplier": 50,
            "symbol": symbol
            },
        "price": amount
    }
    connection.send(json.dumps(request))

    res = connection.recv()
    if res is None:
        print("error")
    data = json.loads(res)
    if "error" in data:
        print("Error:", data)
        
    elif data["msg_type"] == "buy":
        print(data)

    return data

def get_contract(id): #to get contract by id 
    request_open_contract={
        "proposal_open_contract": 1,
        "contract_id": id 
    }
    connection.send(json.dumps(request_open_contract))
    res = connection.recv()
    if res is None:
        print("error")
    else:
        try:
            data = json.loads(res)
            if "error" in data:
                print("Error:", data["error"]["message"])
                
            elif data["msg_type"] == "proposal_open_contract":
                return data
        except json.decoder.JSONDecodeError:    
            print('The string does NOT contain valid JSON')




In [56]:
connection = websocket.create_connection(ws_link)

data = get_symbol()

df = pd.DataFrame(data)[['symbol', 'submarket_display_name']]
df.sort_values('submarket_display_name')


[{'allow_forward_starting': 0, 'display_name': 'AUD Basket', 'display_order': 2, 'exchange_is_open': 1, 'is_trading_suspended': 0, 'market': 'synthetic_index', 'market_display_name': 'Derived', 'pip': 0.001, 'subgroup': 'baskets', 'subgroup_display_name': 'Baskets', 'submarket': 'forex_basket', 'submarket_display_name': 'Forex Basket', 'symbol': 'WLDAUD', 'symbol_type': 'forex_basket'}, {'allow_forward_starting': 0, 'display_name': 'AUD/CAD', 'display_order': 74, 'exchange_is_open': 1, 'is_trading_suspended': 0, 'market': 'forex', 'market_display_name': 'Forex', 'pip': 1e-05, 'subgroup': 'none', 'subgroup_display_name': 'None', 'submarket': 'minor_pairs', 'submarket_display_name': 'Minor Pairs', 'symbol': 'frxAUDCAD', 'symbol_type': 'forex'}, {'allow_forward_starting': 0, 'display_name': 'AUD/CHF', 'display_order': 74, 'exchange_is_open': 1, 'is_trading_suspended': 0, 'market': 'forex', 'market_display_name': 'Forex', 'pip': 1e-05, 'subgroup': 'none', 'subgroup_display_name': 'None', '

,symbol,submarket_display_name
73,OTC_DJI,American indices
53,OTC_SPC,American indices
54,OTC_NDX,American indices
37,OTC_HSI,Asian indices
38,OTC_N225,Asian indices
...,...,...
45,frxNZDUSD,Minor Pairs
44,frxNZDJPY,Minor Pairs
60,frxUSDPLN,Minor Pairs
23,frxEURNZD,Minor Pairs


#### Save the market symbol in array

In [57]:
major_pair_symbols = df[df["submarket_display_name"] == "Major Pairs"]["symbol"].tolist()
crypto_symbols = df[df["submarket_display_name"] == "Cryptocurrencies"]["symbol"].tolist()
frx_basket_symbol = df[df["submarket_display_name"] == "Forex Basket"]["symbol"].tolist()
gold_basket_symbol = df[df["submarket_display_name"] == "Commodities Basket"]["symbol"].tolist()
synthetic_symbol = df[(df["submarket_display_name"] == "Continuous Indices")|(df["submarket_display_name"] == "Crash/Boom Indices")\
                     |(df["submarket_display_name"] == "Step Indices")|(df["submarket_display_name"] == "Jump Indices")]["symbol"].tolist()


In [58]:
contract_types = ["MULTUP","MULTDOWN"]
indices = ["R_10","R_25","R_50","R_75","R_100","1HZ10V","1HZ25V","1HZ50V","1HZ75V","1HZ100V"]
amount = np.arange(10,50,1).tolist()
duration_units = ["d","m","s","h","t"] #Duration unit - s: seconds, m: minutes, h: hours, d: days, t: ticks.


In [60]:
frx_basket_symbol+gold_basket_symbol + crypto_symbols

['WLDAUD', 'WLDEUR', 'WLDGBP', 'WLDUSD', 'WLDXAU', 'cryBTCUSD', 'cryETHUSD']

In [61]:
sell_contract_ids = []

#### Check for next tick implimentation at entry and current tick at sell/exit.
1. Buy n number of random sampling muiltiplier contracts.
   - Save the contract id's for later use
2. Sell all the contracts. (you can manually sell in dtrader or use python)
3. Get all the contract infos.
4. Check for the start time, entry time, exit time, sell time.

#### Buy n number of multiplier contracts

In [ ]:
df_contract = []
while len(df_contract) < 50:
    print(len(df_contract))
    time.sleep(0.7)
    clear_output(wait=True)
    stake = random.choice(amount)
    contract_type = random.choice(contract_types)
    underlying = random.choice(frx_basket_symbol+gold_basket_symbol + crypto_symbols)

    connection = websocket.create_connection(ws_link)
    auth_token(api_token)
    sell_contract_ids.append(buy_contract(stake, underlying, contract_type))

    # buy_contract(stake, underlying, contract_type)
    connection.close()
    sell_contract_ids 
    df_data = {
        'contract_id': [],
        'buy_price': [],
        'contract_type': [],
        'symbol': []
    }
    for item in sell_contract_ids:
        buy_info = item.get('buy', {})
        contract_info = item.get('echo_req', {}).get('parameters', {})  # Get the 'buy' dictionary or an empty dictionary if it doesn't exist
        df_data['contract_id'].append(buy_info.get('contract_id', None))
        df_data['buy_price'].append(buy_info.get('buy_price', None))
        df_data['contract_type'].append(contract_info.get('contract_type', None))
        df_data['symbol'].append(contract_info.get('symbol', None))
    #     print (item)



    df_contract = pd.DataFrame(df_data)
    df_contract = df_contract.dropna().reset_index(drop=True)
    time.sleep(1)
clear_output(wait=True)
df_contract.head()

#### Save the info in pd dataframe

After all the contracts sold (I'm using dtrader to sell all the contracts, but you can use python and call sell endpoints using api to sell also), get the contract infos and save it in dataframes

In [65]:
for i in df_contract.iterrows():
#     clear_output(wait=True)
    time.sleep(1)
    connection = websocket.create_connection(ws_link)
    auth_token(api_token)

    if (np.isnan(i[1]['contract_id'])):
        next
    data = get_contract(int(i[1]["contract_id"]))
    print(data)
    print()

{'account_list': [{'account_category': 'trading', 'account_type': 'binary', 'broker': 'CR', 'created_at': 1719220364, 'currency': 'USD', 'currency_type': 'fiat', 'is_disabled': 0, 'is_virtual': 0, 'landing_company_name': 'svg', 'linked_to': [], 'loginid': 'CR90000013'}, {'account_category': 'trading', 'account_type': 'binary', 'broker': 'VRTC', 'created_at': 1719220364, 'currency': 'USD', 'currency_type': 'fiat', 'is_disabled': 0, 'is_virtual': 1, 'landing_company_name': 'virtual', 'linked_to': [], 'loginid': 'VRTC90000013'}], 'balance': 8391, 'country': 'id', 'currency': 'USD', 'email': 'awdmkm@mail.com', 'fullname': 'Ms QA script awdmkmKSrta', 'is_virtual': 0, 'landing_company_fullname': 'Deriv (SVG) LLC', 'landing_company_name': 'svg', 'linked_to': [], 'local_currencies': {'IDR': {'fractional_digits': 2}}, 'loginid': 'CR90000013', 'preferred_language': 'EN', 'scopes': ['read', 'trade', 'payments', 'admin'], 'upgradeable_landing_companies': ['svg'], 'user_id': 16}
{'echo_req': {'cont

In [ ]:
datums = []
df_tick =pd.DataFrame(columns=['contract_id','contract_type','underlying','date_start','entry_tick_time',\
                              ])
#                                'date_expiry',\
#                                'date_settlement','exit_tick_time','expiry_time','sell_spot_time'])

for i in df_contract.contract_id:
    i = int(i)
    clear_output(wait=True)
    time.sleep(1)
    connection = websocket.create_connection(ws_link)
    auth_token(api_token)

#     if (np.isnan(i[1]['contract_id'])):
#         next
    data = get_contract(i)
#     if (data is not None):     
#         if ('exit_tick' in data['proposal_open_contract']):
#             if (data is None):
#                 next
# #             print(json.dumps(data, indent=4))
#             print(i)
    df_tick.loc[len(df_tick)] = [
                data['proposal_open_contract']["contract_id"],
                data['proposal_open_contract']["contract_type"],
                data['proposal_open_contract']["underlying"],
                data['proposal_open_contract']["date_start"],
                data['proposal_open_contract']["entry_tick_time"],
#                 data['proposal_open_contract']["date_expiry"],
#                 data['proposal_open_contract']["date_settlement"],
#                 data['proposal_open_contract']["exit_tick_time"],
#                 data['proposal_open_contract']["expiry_time"],
#                 data['proposal_open_contract']["sell_spot_time"],

    ]
#             datums.append(datum)
    connection.close()
clear_output(wait=True)
df_tick

In [67]:
df_tick['dt'] = df_tick.date_start - df_tick.entry_tick_time
df_tick['next_tick'] = np.where(df_tick.underlying.str.contains('R_'),df_tick.dt//2,df_tick.dt)

In [73]:
df_tick

,contract_id,contract_type,underlying,date_start,entry_tick_time,dt,next_tick
0,1019,MULTUP,WLDGBP,1719220425,1719220426,-1,-1
1,1039,MULTDOWN,WLDEUR,1719220428,1719220429,-1,-1
2,1059,MULTDOWN,WLDEUR,1719220431,1719220432,-1,-1
3,1079,MULTUP,WLDGBP,1719220436,1719220437,-1,-1
4,1099,MULTUP,WLDXAU,1719220442,1719220444,-2,-2
5,1119,MULTDOWN,WLDEUR,1719220444,1719220445,-1,-1
6,1139,MULTDOWN,WLDXAU,1719220447,1719220448,-1,-1
7,1159,MULTUP,WLDAUD,1719220457,1719220458,-1,-1
8,1179,MULTDOWN,WLDXAU,1719220460,1719220461,-1,-1
9,1199,MULTDOWN,WLDXAU,1719220463,1719220464,-1,-1


In [76]:
df_tick.to_csv('next tick basket crypto 2406.csv',index = False)